In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from math import sqrt

# Asegúrate de importar las clases de tu modelo y el dataset
from preprocess import Dataset
from models_nn import CNN_LSTM_Model, LSTM_Seq2Seq, LSTMModel # <-- Asegúrate de tener LSTMModel si lo usas


In [14]:
# --- CONFIGURACIÓN ---
RUTA_MODELO = 'E:/Dev/Deep-Solar-Grid/saved_models/lstm_simple_no_met.pth'
RUTA_MODELO = 'E:/Dev/Deep-Solar-Grid/saved_models/cnn_lstm_no_met.pth'
RUTA_MODELO = 'E:/Dev/Deep-Solar-Grid/saved_models/seq2seq_lstm_no_met.pth'

RUTA_DATOS_TRAIN = 'E:\Dev\Deep-Solar-Grid\dataset_final_train.csv'
RUTA_DATOS_TEST = 'E:\Dev\Deep-Solar-Grid\dataset_final_test.csv'

MET = 'Sin datos meteorologicos'
# Parámetros que DEBEN COINCIDIR con los del script de entrenamiento
n_pasos = 3
hidden_layer_size = 256
seq2seq = 'seq2seq_lstm' in RUTA_MODELO

# Inicializar el dataset (esto ya procesa los datos de entrenamiento y prueba)
dataset = Dataset(RUTA_DATOS_TRAIN, RUTA_DATOS_TEST, n_pasos=n_pasos, seq2seq=seq2seq)

# Configuración del dispositivo
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, 'xpu') and torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")
print(f"Usando dispositivo: {device}")

# Instanciar el modelo (mismo que en el entrenamiento)
if 'lstm_simple' in RUTA_MODELO:
    model = LSTMModel(input_size=dataset.n_features, hidden_layer_size=hidden_layer_size).to(device)
elif 'cnn_lstm' in RUTA_MODELO:
    model = CNN_LSTM_Model(input_size=dataset.n_features, hidden_layer_size=hidden_layer_size).to(device)
elif 'seq2seq_lstm' in RUTA_MODELO:
    model = LSTM_Seq2Seq(input_size=dataset.n_features, hidden_layer_size=hidden_layer_size, output_sequence_len=10).to(device)
else:
    print("Tipo de modelo no reconocido.")
    exit()

# Cargar los pesos guardados
try:
    model.load_state_dict(torch.load(RUTA_MODELO, map_location=device))
except FileNotFoundError:
    print(f"Error: No se encontró el archivo del modelo en '{RUTA_MODELO}'. Asegúrate de que la ruta sea correcta.")
    exit()
print(f"Modelo cargado desde: {RUTA_MODELO}")

# Poner el modelo en modo de evaluación
model.eval()

# Convertir los datos de prueba a tensores de PyTorch
X_test_tensor = torch.from_numpy(dataset.X_test).float().to(device)
y_test_tensor = torch.from_numpy(dataset.y_test).float().to(device)

print(f"Dimensiones de los datos de prueba: X={X_test_tensor.shape}, y={y_test_tensor.shape}")

Usando dispositivo: cuda
Modelo cargado desde: E:/Dev/Deep-Solar-Grid/saved_models/seq2seq_lstm_no_met.pth
Dimensiones de los datos de prueba: X=torch.Size([2863, 3, 8]), y=torch.Size([2863, 10])


In [15]:
## Evaluación del modelo en todo el conjunto de test
print("\nRealizando predicciones en el conjunto de prueba completo...")

with torch.no_grad():
    predicciones_test_tensor = model(X_test_tensor)

# Mover predicciones a la CPU y convertir a NumPy
predicciones_test_np = predicciones_test_tensor.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

# Ajuste para modelos Seq2Seq
if seq2seq:
    # Si el modelo es Seq2Seq, tomamos la primera predicción de cada secuencia
    predicciones_desescaladas = predicciones_test_np[:, 0]
    # Y también los valores reales correspondientes
    y_reales_desescalados = y_test_np[:, 0]
else:
    # Para modelos no-Seq2Seq, la salida ya es de un solo paso
    predicciones_desescaladas = predicciones_test_np.flatten()
    y_reales_desescalados = y_test_np.flatten()


# Desescalar los valores para que el RMSE sea interpretable en las unidades originales
# (kW en tu caso)
# Para predicciones
predicciones_padded = np.zeros((len(predicciones_desescaladas), dataset.n_features))
predicciones_padded[:, 0] = predicciones_desescaladas
predicciones_final = dataset.scaler.inverse_transform(predicciones_padded)[:, 0]

# Para valores reales
y_reales_padded = np.zeros((len(y_reales_desescalados), dataset.n_features))
y_reales_padded[:, 0] = y_reales_desescalados
y_reales_final = dataset.scaler.inverse_transform(y_reales_padded)[:, 0]

# Calcular el RMSE
rmse = sqrt(mean_squared_error(y_reales_final, predicciones_final))

print(f"\n--- Métricas de rendimiento globales {RUTA_MODELO.split(r'/')[-1].replace('.pth', '')} {MET}---")
print(f"RMSE en el conjunto de prueba: {rmse:.4f}")

# Opcional: Calcular otras métricas como el MAE
mae = np.mean(np.abs(y_reales_final - predicciones_final))
print(f"MAE en el conjunto de prueba: {mae:.4f}")



Realizando predicciones en el conjunto de prueba completo...

--- Métricas de rendimiento globales seq2seq_lstm_no_met Sin datos meteorologicos---
RMSE en el conjunto de prueba: 7.4612
MAE en el conjunto de prueba: 3.9858
